##### IMPORTAR AS PLANILHAS

In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from resolve_path import ajuste_path

##### IMPORTANDO PLANILHA

In [ ]:
path_iw = "data/util/os/"
path_iw = ajuste_path(path_iw)
df_agrupado = pd.read_csv(path_iw + "os_entrada.csv")
df = pd.read_csv(path_iw + "IW49_Executadas_processado.csv")

In [ ]:
df["Data fim"] = pd.to_datetime(df["Data fim"])
df["Data inicio"] = pd.to_datetime(df["Data inicio"])
df["Local de instalação"] = df["Local de instalação"].astype(str)

##### CRIANDO DATAFRAME SEM INCLUIR DATAS FINAIS ANTERIORES A DATAS INICIAIS

In [ ]:
# Removendo linhas onde a data final é superior a data inicial
df_sem_linhas_irregulares = df.loc[df["Data fim"]
                                   >= df["Data inicio"]]

num_linhas_removidas = df.shape[0] - df_sem_linhas_irregulares.shape[0]
print("Linhas removidas:", num_linhas_removidas)

##### FUNÇÃO QUE ADD COLUNA DE HH POR MES E QUANTIDADE DE MESES PASSADOS DURANTE A OS

In [ ]:
# Soma dos meses, em funcao da quantidade de anos que se passaram e meses, entre o inicio e fim da OS


def duracaoHH(linha):
    linha["Meses"] = (
        (linha["Data fim"].year - linha["Data inicio"].year) * 12
        + (linha["Data fim"].month - linha["Data inicio"].month)
    ) + 1
    linha["HH por mes"] = linha["HH final"] / linha["Meses"]
    return linha


df = df_sem_linhas_irregulares.apply(duracaoHH, axis=1)

##### FUNÇÃO QUE TESTA O DATAFRAME DE ENTRADA

In [ ]:
def testaDfAgrupado(anoTeste, mesTeste, localTeste):
    dataInicioTeste = pd.to_datetime(
        f"{anoTeste}-{mesTeste}-01") + pd.DateOffset(months=1) - pd.DateOffset(days=1)
    dataFimTeste = pd.to_datetime(f"{anoTeste}-{mesTeste:02d}-01")

    # Filtrar o DataFrame original para o todas as OSs que estavam ocorrendo durante anoTeste e mesTeste
    testeDf = df.loc[
        (df["Data inicio"] <= dataInicioTeste)
        & (df["Data fim"] >= dataFimTeste)
        & (df["Local de instalação"] == localTeste)
    ]

    # Filtrar o DataFrame agregado para o mesmo ano, mês e local
    testeDfAgrupado = df_agrupado.loc[
        (df_agrupado["Ano Mes"] == f"{anoTeste}-{mesTeste:02d}")
        & (df_agrupado["Local de instalação"] == localTeste)
    ]

    # Soma a coluna 'HH' do DataFrame original filtrado
    somaHH = testeDf["HH por mes"].sum()

    # Obter o valor de 'HH final' do DataFrame agregado filtrado
    hhFinalAgrupado = testeDfAgrupado["HH"].sum()

    # Testar se os valores são iguais. Foi realizada uma operação de diferença para levar em conta erros de precisão do float.
    diferenca = abs(somaHH - hhFinalAgrupado)
    if diferenca < 1e-6:
        diferenca = 0.0
    return diferenca

##### LOOP QUE REALIZA OS TESTES

In [ ]:
for i in range(100):
    anoTeste = random.randint(2000, 2024)
    mesTeste = random.randint(1, 12)
    localTeste = random.choice(df["Local de instalação"].unique())
    print(f"Ano: {anoTeste}, Mês: {mesTeste}, Local: {localTeste}")
    resultado = testaDfAgrupado(anoTeste, mesTeste, localTeste)
    assert resultado == 0.0, f"Teste falhou, diferença de HH de {resultado}"
    print("Teste passou com sucesso")